<a href="https://colab.research.google.com/github/DSF-Kotaro/blog/blob/main/%E5%9B%B3%E8%A7%A3%E9%80%9F%E7%BF%92DeepLearning_015.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


In [6]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [7]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [8]:
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [9]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

F
i
r
s
t


In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [12]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [13]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [14]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


In [15]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [16]:
vocab_size = len(vocab)

embedding_dim = 256

rnn_units = 1024

In [17]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [18]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [19]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [21]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [22]:
sampled_indices

array([ 2, 56, 30,  8, 61,  6, 63, 57,  9, 26, 29, 12, 29, 63, 63, 34, 27,
       23, 24,  7, 13, 29, 42, 43, 38,  7, 63, 30, 25, 61, 28,  3, 53, 35,
       25,  7, 42, 31, 31, 11,  1,  0, 43, 45,  6, 15, 10, 21, 29,  5, 60,
       20, 59, 35, 18, 34, 28, 62, 27, 33, 21, 55, 58, 22, 38, 48, 35, 24,
       51,  7, 30, 13, 57, 25, 39, 60, 42, 20, 40, 46,  8, 18, 53, 43, 64,
       13, 54, 23,  8, 47, 42,  1, 45, 37, 63, 43, 30, 44, 15, 44])

In [23]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 " say true: as Hereford's love, so his;\nAs theirs, so mine; and all be as it is.\n\nNORTHUMBERLAND:\nMy "

Next Char Predictions: 
 "!rR.w,ys3NQ?QyyVOKL-AQdeZ-yRMwP$oWM-dSS; \neg,C:IQ'vHuWFVPxOUIqtJZjWLm-RAsMavdHbh.FoezApK.id gYyeRfCf"


In [24]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.174324


In [25]:
model.compile(optimizer='adam', loss=loss)

In [26]:
# チェックポイントが保存されるディレクトリ
checkpoint_dir = './training_checkpoints'
# チェックポイントファイルの名称
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [27]:
EPOCHS=10

In [28]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 11s 52ms/step - loss: 3.2097
Epoch 2/10
172/172 [==============================] - 10s 52ms/step - loss: 2.0411
Epoch 3/10
172/172 [==============================] - 10s 53ms/step - loss: 1.7323
Epoch 4/10
172/172 [==============================] - 10s 53ms/step - loss: 1.5603
Epoch 5/10
172/172 [==============================] - 10s 54ms/step - loss: 1.4657
Epoch 6/10
172/172 [==============================] - 10s 54ms/step - loss: 1.3963
Epoch 7/10
172/172 [==============================] - 10s 54ms/step - loss: 1.3453
Epoch 8/10
172/172 [==============================] - 10s 55ms/step - loss: 1.3037
Epoch 9/10
172/172 [==============================] - 10s 55ms/step - loss: 1.2657
Epoch 10/10
172/172 [==============================] - 11s 56ms/step - loss: 1.2319


In [29]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [30]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [32]:
def generate_text(model, start_string):

  num_generate = 1000

  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []

  temperature = 1.0

  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)

      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [33]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: Nurse!
Nay, get thee, or if you?
Had not he hath an oath but nobly cengman's haspeth war
that thou wilt have him baxhaline.
Now, sir, by Than's eyes fastel old,
That cannot be forget i' the suitor,
Northumberling
speaks thou hadst kepore thine eyes;
No, nothing budgen'd of my forseman.

HERMIONE:
Most he to be heell ying, sir.

VINCENTIO:

TRANIO:
Nay, hear you so? O wood and truth!
O, who bloody Captifts yourself, which have done low out.

TRANIO:
Nay, asains her doth come to gie, and strike
My fian I' the friendly father?

LUCENTIO:
A grace!

HERMIONE:
Northumberland horse and my sin pe
dispersedomer, and therein time and me:
If they say 'I'll keep. Death, wisdom.
March to thy knift and courtesy, as stands?
Are you not, take that all the damn
To this wry 'twixt so scould have done'er strike rests,
Sevately hours: no dreddicinn have
with her, though by his tadster kind by many. Yes, keep your heads,
That look'd on and stand, what an inflicted of
himself, for we are.
Let not how

In [34]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [35]:
optimizer = tf.keras.optimizers.Adam()

In [36]:
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [37]:
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  hidden = model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.173957824707031
Epoch 1 Batch 100 Loss 2.3314778804779053
Epoch 1 Loss 2.1555
Time taken for 1 epoch 11.281167268753052 sec

Epoch 2 Batch 0 Loss 2.1367037296295166
Epoch 2 Batch 100 Loss 1.9241223335266113
Epoch 2 Loss 1.8167
Time taken for 1 epoch 10.425826072692871 sec

Epoch 3 Batch 0 Loss 1.7992395162582397
Epoch 3 Batch 100 Loss 1.6966116428375244
Epoch 3 Loss 1.5534
Time taken for 1 epoch 10.46082091331482 sec

Epoch 4 Batch 0 Loss 1.5998129844665527
Epoch 4 Batch 100 Loss 1.534644603729248
Epoch 4 Loss 1.5422
Time taken for 1 epoch 10.56141471862793 sec

Epoch 5 Batch 0 Loss 1.4495196342468262
Epoch 5 Batch 100 Loss 1.4283349514007568
Epoch 5 Loss 1.4395
Time taken for 1 epoch 10.753752946853638 sec

Epoch 6 Batch 0 Loss 1.37766432762146
Epoch 6 Batch 100 Loss 1.4204047918319702
Epoch 6 Loss 1.4169
Time taken for 1 epoch 10.712035655975342 sec

Epoch 7 Batch 0 Loss 1.3377376794815063
Epoch 7 Batch 100 Loss 1.3669263124465942
Epoch 7 Loss 1.3799
Time taken